<a href="https://colab.research.google.com/github/rodrigoromanguzman/Actividades_Aprendizaje-/blob/main/MNA_IAyAA_semana_10_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   Adrián Alejandro Dávila González - A01039334
*   Juan Antonio Melendres Villa - A00369017
*   Andrea Margarita Osorio González - A01104776
*   Rodrigo Ildefonso Román Guzmán - A01794225
*   Jeanette Ríos Martínez - A01688888

En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [209]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import requests
import io

#Librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

#Modelos
from sklearn.decomposition import TruncatedSVD


#**Ejercicio-1.**

Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega sus primeros 10 renglones.


In [210]:
#Descargamos los datos de las evaluaciones realizadas

url = 'https://drive.google.com/file/d/1W7h2j_fLf7SI37FiBRS5_gXUlvLeJvLi/view?usp=sharing'
path='https://drive.google.com/uc?id=' + url.split('/')[-2]
response = requests.get(path)
contents = response.text
lines = contents.split('\n')
userRestaurantIds = pd.read_csv(io.StringIO('\n'.join(lines)), sep=',', header='infer')
userRestaurantIds.head(10)

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2
5,U1068,132740,0,0,0
6,U1068,132663,1,1,1
7,U1068,132732,0,0,0
8,U1068,132630,1,1,1
9,U1067,132584,2,2,2


In [211]:
#Descargamos los datos de los
#De esta base de datos vamos a tomar el nombre del restaurante

url = 'https://drive.google.com/file/d/1nAH4knCXMhAhktVGip0ftzTRC6206Np8/view?usp=sharing'
path='https://drive.google.com/uc?id=' + url.split('/')[-2]
response = requests.get(path)
contents = response.text
lines = contents.split('\n')
restaurantSpecs = pd.read_csv(io.StringIO('\n'.join(lines)), sep=',', header='infer')
restaurantSpecs.head(10)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none
5,135040,22.135617,-100.969709,0101000020957F00001B552189B84A58C15A2AAEFD2CA2...,Restaurant los Compadres,Camino a Simon Diaz 155 Centro,San Luis Potosi,SLP,Mexico,?,...,Wine-Beer,none,informal,no_accessibility,high,?,familiar,f,closed,none
6,132732,23.754357,-99.171288,0101000020957F00008A20E615808157C16272FECBF84F...,Taqueria EL amigo,Calle Mezquite Fracc Framboyanes,Cd Victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,none,casual,completely,low,?,familiar,f,open,none
7,132875,22.149901,-100.993779,0101000020957F00008A2A0747DE4758C11EB31D2A31A8...,shi ro ie,?,?,?,?,?,...,Wine-Beer,section,informal,no_accessibility,high,?,familiar,t,open,Internet
8,132609,23.760268,-99.165865,0101000020957F0000A478418BBA8057C133851EB22C4E...,Pollo_Frito_Buenos_Aires,tampico,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,not permitted,informal,completely,low,?,quiet,t,closed,none
9,135082,22.151448,-100.915099,0101000020957F0000A29FAF95CD4958C1FEEEBB73A991...,la Estrella de Dimas,Villa de Pozos 192 Villa de Pozos,San Luis Potosi,SLP,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,?,familiar,f,closed,none


Vamos a juntar la columna de los nombres con nuestro principal Dataframe que contiene el id de los usuarios y de los restaurantes con el nombre del restaurante

In [212]:
realtionMatrix = pd.merge(userRestaurantIds, restaurantSpecs[['name','placeID']], on='placeID')   # dataFrame

realtionMatrix.head()

,userID,placeID,rating,food_rating,service_rating,name
0,U1077,135085,2,2,2,Tortas Locas Hipocampo
1,U1108,135085,1,2,1,Tortas Locas Hipocampo
2,U1081,135085,1,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,2,Tortas Locas Hipocampo
4,U1134,135085,2,1,2,Tortas Locas Hipocampo


Visuzalizamos la dimension de nuestra tabla de relacion

In [213]:
realtionMatrix.shape

(1161, 6)

A partir de esto podemos generar nuestra matriz de utilidad

In [214]:
utilityMatrix = realtionMatrix.pivot_table(values='rating', index='userID', columns='name', fill_value=0)
utilityMatrix.head(10)


name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


Tamaño de nuestra matriz de utilidad

In [215]:
utilityMatrix.shape

(138, 129)

Revisamos la relacion de datos nulos con la totalidad de datos

In [216]:
print('Total de elementos de la matriz de utilidad: %d' % (utilityMatrix.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(utilityMatrix)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(utilityMatrix) / utilityMatrix.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 907
Porcentaje de elementos diferentes de cero: 5.1%


In [217]:
X = utilityMatrix.T     # Tomamos la traspuesta para que los renglones sean los restaurantes y las columnas los usuarios.
X.shape

(129, 138)

In [218]:
X

userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,2,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tacos de barbacoa enfrente del Tec,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tacos de la estacion,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tacos los volcanes,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


#**Ejercicio-2.**

Con el objetivo de obtener la información con respecto a los vectores latentes de los restaurantes, obtener dicha factorización SVD utilizando todas las componentes posibles, es decir, usando todos sus valores singulares. En particular indica la cantidad de valores singulares obtenidos.

In [219]:

# Calculamos el SVD con una cantidad "IGUAL" al numero de caracteristicas
# Este numero es tambien el maximo debido a que no puede exceder la cantidad
# de caracteristicas

SVD = TruncatedSVD(n_components=129, random_state=1)
SVD.fit(X)
completeSVD = SVD.fit_transform(X)
print('Cantidad de valores singulares obtenidos: ',len(SVD.singular_values_))

Cantidad de valores singulares obtenidos:  129


#**Ejercicio-3.**

Usando la métrica de correlación de Pearson, obtener las 10 mejores recomendaciones de este modelo no supervisado con base a la información de alguien que evaluó bien de manera general al “Subway”. Despliega también el valor numérico de la correlación de Pearson de cada una de estas 10 recomendaciones. NOTA: No debe estar incluido “Subway” dentro de las recomendaciones.

In [220]:
# Obtengamos la matriz de correlación de Pearson entre todas las
# variables latentes de los restaurantes:
# Por default np.corrcoef usa la correlaion de Pearson
completCorrelationMatrix = np.corrcoef(completeSVD, )
completCorrelationMatrix.shape

(129, 129)

In [221]:
# Indice de subway  en la matriz de utilidad:
subway = "Subway"

names = utilityMatrix.columns
namesList = list(names)
idSubway = namesList.index(subway)

idSubway

88

In [222]:
# Veamos la correlación de Subway con todos los demas restaurantes:
correlationRecomendations = completCorrelationMatrix[idSubway]
print(correlationRecomendations.shape)
print(correlationRecomendations)

(129,)
[-7.69028369e-04 -7.87822168e-05 -5.27330975e-03  8.58511349e-04
 -3.03891148e-03  3.97495832e-05 -1.54820208e-03 -5.29347145e-03
  7.27520383e-04 -1.55098452e-03 -1.84483094e-03  3.01090491e-01
 -3.06938616e-05 -3.66639157e-03 -2.28024815e-03 -1.70971180e-03
  2.66961340e-03  4.58467365e-04 -2.11449533e-03 -6.20445134e-03
  3.34268563e-03  1.16598607e-01  1.87080102e-03  2.53682556e-03
 -5.48801141e-03 -2.91288266e-03  2.02825342e-03 -1.36607265e-03
 -2.01695899e-03  1.12058244e-01 -2.36235727e-03  6.52159239e-04
  3.53495770e-03 -3.62793330e-03  1.69065407e-03 -4.51837401e-03
 -2.09301342e-03  2.34634298e-01 -3.80413494e-03 -5.11845710e-04
 -2.27148704e-03  3.88178002e-01 -2.07427108e-03 -2.29428835e-03
  1.32565455e-03  3.01002479e-01  1.71433518e-01 -3.29715354e-03
 -4.74480596e-03 -8.02527884e-04 -2.65071452e-03  1.12018889e-01
 -3.91080724e-03 -2.06369812e-03  1.16008629e-01  2.27855804e-01
  4.28342495e-04 -8.69739761e-04  6.09520875e-01 -6.43364064e-03
  4.25467169e-01 -

In [223]:
# Ponemos un limite superior de 0.99 para que no correlacione tambien el propio
# Subway
upLimit = 0.99
lowLimit = 0.23
selectedNamesFull = names[(correlationRecomendations > lowLimit) & (correlationRecomendations < upLimit)]
sortedData = sorted(zip(selectedNamesFull, correlationRecomendations[(correlationRecomendations > lowLimit) & (correlationRecomendations < upLimit)]), key=lambda x: x[1], reverse=True)

maxNameLength = max(len(name) for name, _ in sortedData)

for name, correlation in sortedData:
    print(f"Nombre: {name:{maxNameLength}} Correlacion: {correlation:.2f}")

Nombre: Restaurant Teely                                     Correlacion: 0.61
Nombre: Vips                                                 Correlacion: 0.51
Nombre: Rincon del Bife                                      Correlacion: 0.43
Nombre: Restaurant and Bar and Clothesline Carlos N Charlies Correlacion: 0.43
Nombre: Sanborns Casa Piedra                                 Correlacion: 0.42
Nombre: Mariscos Tia Licha                                   Correlacion: 0.39
Nombre: Chilis Cuernavaca                                    Correlacion: 0.30
Nombre: Mikasa                                               Correlacion: 0.30
Nombre: Restaurant los Pinos                                 Correlacion: 0.29
Nombre: Log Yin                                              Correlacion: 0.23


#**Ejercicio-4.**

In [224]:
num_sv = 30  # Seleccionamos el numero de vectores singulares
print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Cantidad de información simplificada con los primeros 30 vectores singulares:
47.0%


Disminuye ahora la dimensión de la factorización con un número de componentes tal, que las recomendaciones con esta menor cantidad de información coincidan en al menos 9 de las 10 recomendaciones dadas con todos los valores singulares. Indica la cantidad de valores singulares encontrado y la cantidad de información que contienen.

In [225]:
#Definimos una matriz con una cantidad de 20 vectores solamente
SVD = TruncatedSVD(n_components=num_sv, random_state=1)
truncatedSVD = SVD.fit_transform(X)
truncatedSVD.shape

(129, 30)

In [226]:
correlationMatrixReduced = np.corrcoef(truncatedSVD, )
correlationMatrixReduced

array([[ 1.        ,  0.04514676, -0.12184271, ..., -0.03207761,
        -0.04330619, -0.03391959],
       [ 0.04514676,  1.        ,  0.11128586, ...,  0.05446879,
        -0.01017322,  0.05816478],
       [-0.12184271,  0.11128586,  1.        , ...,  0.13475348,
        -0.07433494, -0.07981804],
       ...,
       [-0.03207761,  0.05446879,  0.13475348, ...,  1.        ,
        -0.0038355 , -0.03708557],
       [-0.04330619, -0.01017322, -0.07433494, ..., -0.0038355 ,
         1.        ,  0.95449016],
       [-0.03391959,  0.05816478, -0.07981804, ..., -0.03708557,
         0.95449016,  1.        ]])

In [227]:
# Correlación de subway con los demas restaurantes
correlationRecommendReduced = correlationMatrixReduced[idSubway]
print(correlationRecommendReduced.shape)
print(correlationRecommendReduced)

(129,)
[-0.03602434 -0.23481722 -0.10040932  0.01793079 -0.05869319 -0.01018214
 -0.02981529 -0.00979703 -0.07780665 -0.01960738 -0.02048897  0.73833296
 -0.00183436 -0.05513242 -0.02048907 -0.03312135 -0.04385997 -0.00448809
 -0.00452969 -0.06944352 -0.10483174  0.04644896  0.02665231  0.03692843
 -0.02541816 -0.06192549 -0.02121939 -0.02528444 -0.00479267  0.01102943
 -0.04914154  0.01261178  0.07348972 -0.05434129  0.03448823 -0.0720798
 -0.03822253  0.74822676  0.32265848 -0.02557008 -0.03018736  0.5619614
  0.03121097 -0.03346253  0.00386958  0.58552241  0.58271994 -0.05898611
 -0.0410869  -0.02237872 -0.03217557  0.08096765 -0.05669525 -0.02426094
  0.03279983  0.20456721 -0.00716231 -0.0548328   0.84158877 -0.10212661
  0.84490718 -0.0338506  -0.01383015 -0.05556778  0.52099215 -0.02211963
 -0.00997623 -0.0440739  -0.03686922 -0.03302898 -0.12771961  0.01462413
 -0.0244195  -0.01858985  0.04182561 -0.05927814 -0.07453949 -0.02407751
  0.03937527 -0.01467342 -0.04752109 -0.034294

In [228]:
# Subway
lowLimit = 0.53
upLimit = 0.99

selectedNamesReduced = names[(correlationRecommendReduced > lowLimit) & (correlationRecommendReduced < upLimit)]
sorted_data = sorted(zip(selectedNamesReduced, correlationRecommendReduced[(correlationRecommendReduced > lowLimit) & (correlationRecommendReduced < upLimit)]), key=lambda x: x[1], reverse=True)

maxNameLength = max(len(name) for name, _ in sorted_data)

for name, correlation in sorted_data:
    print(f"Nombre: {name:{maxNameLength}} Correlacion: {correlation:.2f}")

Nombre: Rincon del Bife                                      Correlacion: 0.92
Nombre: Restaurant and Bar and Clothesline Carlos N Charlies Correlacion: 0.84
Nombre: Restaurant Teely                                     Correlacion: 0.84
Nombre: Vips                                                 Correlacion: 0.80
Nombre: Log Yin                                              Correlacion: 0.75
Nombre: Chilis Cuernavaca                                    Correlacion: 0.74
Nombre: Sanborns Casa Piedra                                 Correlacion: 0.70
Nombre: Mikasa                                               Correlacion: 0.59
Nombre: Paniroles                                            Correlacion: 0.58
Nombre: Mariscos Tia Licha                                   Correlacion: 0.56


Comparacion de los restaurantes encontrados con los todos los vectores y con solo num_sv

In [229]:

# Convertimos en un set
setReduced = set(selectedNamesReduced)
setFull = set(selectedNamesFull)

# Encontramos elementos comunes entre las listas
commonElements = setReduced.intersection(setFull)

# Encontramos elementos unicos de cada lista
uniqueToReduced = setReduced.difference(setFull)
uniqueToFull = setFull.difference(setReduced)

# Find elements present in one set but not the other
symmetricDifference = setReduced.symmetric_difference(setFull)

# Print the results
print("Elementos en comun:")
print(commonElements)

print("\nElementos pertenecientes solo a selectedNamesReduced:")
print(uniqueToReduced)

print("\nElementos pertenecientes solo a selectedNamesFull:")
print(uniqueToFull)

print("\nElementos presentes en uno pero no en ambos:")
print(symmetricDifference)

Elementos en comun:
{'Rincon del Bife', 'Sanborns Casa Piedra', 'Mikasa', 'Log Yin', 'Vips', 'Chilis Cuernavaca', 'Restaurant Teely', 'Mariscos Tia Licha', 'Restaurant and Bar and Clothesline Carlos N Charlies'}

Elementos pertenecientes solo a selectedNamesReduced:
{'Paniroles'}

Elementos pertenecientes solo a selectedNamesFull:
{'Restaurant los Pinos'}

Elementos presentes en uno pero no en ambos:
{'Paniroles', 'Restaurant los Pinos'}


#**Ejercicio-5.**

Repite el ejercicio con el restaurante “"tacos de barbacoa enfrente del Tec". Incluye tus conclusiones.

In [230]:
# Indice de  en la matriz de utilidad:

tacosTec = "tacos de barbacoa enfrente del Tec"

names = utilityMatrix.columns
namesList = list(names)
idTacosTec = namesList.index(tacosTec)

idTacosTec

124

In [231]:
correlationRecomendations = completCorrelationMatrix[idTacosTec]
print(correlationRecomendations.shape)
print(correlationRecomendations)

(129,)
[ 1.90365985e-03  1.95018219e-04  1.30535991e-02 -2.12516683e-03
  7.52254923e-03 -9.83964813e-05  3.83243356e-03  1.31035076e-02
  4.07794920e-01  3.83932123e-03  4.56671136e-03  1.56152873e-03
  7.59798653e-05  9.07581917e-03  5.64454708e-03  4.23223620e-03
 -6.60838540e-03 -1.13489430e-03  5.23424104e-03  1.53585555e-02
 -8.27451453e-03  4.82434184e-03 -4.63099792e-03 -6.27968115e-03
  1.48442849e-01  1.91120451e-01 -5.02075704e-03  3.38158871e-03
  4.99279870e-03  5.56810594e-03  5.84780076e-03 -1.61436094e-03
 -8.75046657e-03  8.98061922e-03 -4.18506052e-03  1.11848243e-02
  2.58479385e-01  1.27230718e-02  9.41679037e-03  1.26702755e-03
  5.62285977e-03  5.74403685e-03  5.13466958e-03  5.67930233e-03
 -3.28153738e-03  1.83689220e-03  1.11393805e-02  8.16180397e-03
  1.17453360e-02  1.98658485e-03  6.56160293e-03  5.69993965e-03
  9.68084797e-03  5.10849717e-03  6.85473722e-03 -3.44059875e-03
 -1.06032292e-03  2.15296175e-03  1.33109496e-04  1.59258928e-02
  6.48831289e-03  

In [232]:
# Ponemos un limite superior de 0.99 para que no correlacione tambien el propio
# tacos del tec
upLimit = 0.99
downLimit = 0.19
selectedNames = names[(correlationRecomendations > downLimit) & (correlationRecomendations < upLimit)]
sortedData = sorted(zip(selectedNames, correlationRecomendations[(correlationRecomendations > downLimit) & (correlationRecomendations < upLimit)]), key=lambda x: x[1], reverse=True)

maxNameLength = max(len(name) for name, _ in sortedData)

for name, correlation in sortedData:
    print(f"Nombre: {name:{maxNameLength}} Correlacion: {correlation:.2f}")

Nombre: vips                           Correlacion: 0.82
Nombre: tacos abi                      Correlacion: 0.67
Nombre: little pizza Emilio Portes Gil Correlacion: 0.65
Nombre: Taqueria EL amigo              Correlacion: 0.44
Nombre: puesto de gorditas             Correlacion: 0.41
Nombre: Carreton de Flautas y Migadas  Correlacion: 0.41
Nombre: Little Cesarz                  Correlacion: 0.26
Nombre: palomo tec                     Correlacion: 0.20
Nombre: Hamburguesas La perica         Correlacion: 0.19


In [233]:
num_sv = 30   # En particular los 30 primeros vectores y valores singulares de mayor magnitud tienen esta cantidad de información.

print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Cantidad de información simplificada con los primeros 30 vectores singulares:
0.0%


In [234]:
#Definimos una matriz con una cantidad de 10 vectores solamente
SVD = TruncatedSVD(n_components=num_sv, random_state=1)
truncatedSVD = SVD.fit_transform(X)
truncatedSVD.shape

(129, 30)

In [235]:
correlationMatrixReduced = np.corrcoef(truncatedSVD, )
correlationMatrixReduced

array([[ 1.        ,  0.04514676, -0.12184271, ..., -0.03207761,
        -0.04330619, -0.03391959],
       [ 0.04514676,  1.        ,  0.11128586, ...,  0.05446879,
        -0.01017322,  0.05816478],
       [-0.12184271,  0.11128586,  1.        , ...,  0.13475348,
        -0.07433494, -0.07981804],
       ...,
       [-0.03207761,  0.05446879,  0.13475348, ...,  1.        ,
        -0.0038355 , -0.03708557],
       [-0.04330619, -0.01017322, -0.07433494, ..., -0.0038355 ,
         1.        ,  0.95449016],
       [-0.03391959,  0.05816478, -0.07981804, ..., -0.03708557,
         0.95449016,  1.        ]])

In [236]:
# Veamos la correlación de esta película, con todas las que se tienen en el catálogo:

correlationRecommendReduced = correlationMatrixReduced[idTacosTec]
print(correlationRecommendReduced.shape)
print(correlationRecommendReduced)

(129,)
[-3.91455862e-02  5.43357026e-02 -7.88945046e-02  1.62084049e-02
  5.26084062e-03 -9.75231376e-04 -1.58303009e-02  9.77434366e-01
  9.94201749e-01 -2.18792636e-03 -2.19579125e-02 -2.35606387e-02
  1.03183221e-02 -4.71219351e-02 -1.19680155e-02 -2.97884637e-02
  3.98592944e-02 -4.38660537e-03  7.19640222e-04 -5.15679172e-02
  4.10589030e-02 -4.12789635e-02  1.19471937e-02  2.23837542e-02
  9.94330629e-01  9.94281633e-01 -1.71522669e-04 -2.75240680e-02
  3.23705244e-03 -4.21740601e-02 -3.72438122e-02  1.37476078e-02
  4.77200462e-02 -4.70200214e-02  2.74162641e-02 -7.04678648e-02
  9.98452527e-01 -5.15948120e-02 -4.37676608e-02 -2.42117105e-02
 -2.53940647e-02 -3.70305200e-02  7.99784390e-03 -2.32270799e-02
  1.05079602e-03 -3.53507662e-02 -5.11563662e-02 -6.40105351e-02
  9.90659385e-01 -1.78594797e-02 -3.27542553e-02 -4.34482199e-02
 -4.01668183e-02 -1.89921739e-02 -4.38114132e-02  7.59269553e-03
  8.98918628e-03 -4.40938749e-02 -5.35777667e-02 -6.63260958e-02
 -3.25518098e-02 -

In [237]:
# Tacos
lowLimit = 0.35
upLimit = 0.99

selected_names = names[(correlationRecommendReduced > lowLimit) & (correlationRecommendReduced < upLimit)]
sorted_data = sorted(zip(selected_names, correlationRecommendReduced[(correlationRecommendReduced > lowLimit) & (correlationRecommendReduced < upLimit)]), key=lambda x: x[1], reverse=True)

max_name_length = max(len(name) for name, _ in sorted_data)

for name, correlation in sorted_data:
    print(f"Nombre: {name:{max_name_length}} Correlacion: {correlation:.2f}")

Nombre: churchs                               Correlacion: 0.99
Nombre: Carnitas Mata  Calle 16 de Septiembre Correlacion: 0.98
Nombre: TACOS EL GUERO                        Correlacion: 0.97
Nombre: tacos de la estacion                  Correlacion: 0.97
Nombre: cafe ambar                            Correlacion: 0.96
Nombre: tortas hawai                          Correlacion: 0.96
Nombre: TACOS CORRECAMINOS                    Correlacion: 0.94
Nombre: la perica hamburguesa                 Correlacion: 0.94


#**Ejercicio-6.**

Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluación dada a la comida (food_rating). Incluye tus conclusiones.

In [238]:
realtionMatrix = pd.merge(userRestaurantIds, restaurantSpecs[['name','placeID']], on='placeID')   # dataFrame

realtionMatrix.head()

,userID,placeID,rating,food_rating,service_rating,name
0,U1077,135085,2,2,2,Tortas Locas Hipocampo
1,U1108,135085,1,2,1,Tortas Locas Hipocampo
2,U1081,135085,1,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,2,Tortas Locas Hipocampo
4,U1134,135085,2,1,2,Tortas Locas Hipocampo


In [239]:
utilityMatrix = realtionMatrix.pivot_table(values='food_rating', index='userID', columns='name', fill_value=0)
utilityMatrix.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


Revisamos relacion de datos nulos con la totalidad de datos

In [240]:
print('Total de elementos de la matriz de utilidad: %d' % (utilityMatrix.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(utilityMatrix)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(utilityMatrix) / utilityMatrix.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 895
Porcentaje de elementos diferentes de cero: 5.0%


In [241]:
SVD = TruncatedSVD(n_components=129, random_state=1)
SVD.fit(X)
completeSVD = SVD.fit_transform(X)
print('Cantidad de valores singulares obtenidos: ',len(SVD.singular_values_))

Cantidad de valores singulares obtenidos:  129


In [242]:
completCorrelationMatrix = np.corrcoef(completeSVD, )
completCorrelationMatrix.shape

(129, 129)

In [243]:
subway = "Subway"

names = utilityMatrix.columns
namesList = list(names)
idSubway = namesList.index(subway)

idSubway

88

In [244]:
correlationRecomendations = completCorrelationMatrix[idSubway]
print(correlationRecomendations.shape)
print(correlationRecomendations)

(129,)
[-7.69028369e-04 -7.87822168e-05 -5.27330975e-03  8.58511349e-04
 -3.03891148e-03  3.97495832e-05 -1.54820208e-03 -5.29347145e-03
  7.27520383e-04 -1.55098452e-03 -1.84483094e-03  3.01090491e-01
 -3.06938616e-05 -3.66639157e-03 -2.28024815e-03 -1.70971180e-03
  2.66961340e-03  4.58467365e-04 -2.11449533e-03 -6.20445134e-03
  3.34268563e-03  1.16598607e-01  1.87080102e-03  2.53682556e-03
 -5.48801141e-03 -2.91288266e-03  2.02825342e-03 -1.36607265e-03
 -2.01695899e-03  1.12058244e-01 -2.36235727e-03  6.52159239e-04
  3.53495770e-03 -3.62793330e-03  1.69065407e-03 -4.51837401e-03
 -2.09301342e-03  2.34634298e-01 -3.80413494e-03 -5.11845710e-04
 -2.27148704e-03  3.88178002e-01 -2.07427108e-03 -2.29428835e-03
  1.32565455e-03  3.01002479e-01  1.71433518e-01 -3.29715354e-03
 -4.74480596e-03 -8.02527884e-04 -2.65071452e-03  1.12018889e-01
 -3.91080724e-03 -2.06369812e-03  1.16008629e-01  2.27855804e-01
  4.28342495e-04 -8.69739761e-04  6.09520875e-01 -6.43364064e-03
  4.25467169e-01 -

In [245]:
# Ponemos un limite superior de 0.99 para que no correlacione tambien el propio
# Subway
lowLimit = 0.23
upLimit = 0.99

selectedNames = names[(correlationRecomendations > lowLimit) & (correlationRecomendations < upLimit)]
sortedData = sorted(zip(selectedNames, correlationRecomendations[(correlationRecomendations > lowLimit) & (correlationRecomendations < upLimit)]), key=lambda x: x[1], reverse=True)

maxNameLength = max(len(name) for name, _ in sortedData)

for name, correlation in sortedData:
    print(f"Nombre: {name:{maxNameLength}} Correlacion: {correlation:.2f}")

Nombre: Restaurant Teely                                     Correlacion: 0.61
Nombre: Vips                                                 Correlacion: 0.51
Nombre: Rincon del Bife                                      Correlacion: 0.43
Nombre: Restaurant and Bar and Clothesline Carlos N Charlies Correlacion: 0.43
Nombre: Sanborns Casa Piedra                                 Correlacion: 0.42
Nombre: Mariscos Tia Licha                                   Correlacion: 0.39
Nombre: Chilis Cuernavaca                                    Correlacion: 0.30
Nombre: Mikasa                                               Correlacion: 0.30
Nombre: Restaurant los Pinos                                 Correlacion: 0.29
Nombre: Log Yin                                              Correlacion: 0.23


In [246]:
num_sv = 30   # En particular los 30 primeros vectores y valores singulares de mayor magnitud tienen esta cantidad de información.

print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Cantidad de información simplificada con los primeros 30 vectores singulares:
47.0%


In [247]:
#Definimos una matriz con una cantidad de 10 vectores solamente
SVD = TruncatedSVD(n_components=num_sv, random_state=1)
truncatedSVD = SVD.fit_transform(X)
truncatedSVD.shape

(129, 30)

In [248]:
correlationMatrixReduced = np.corrcoef(truncatedSVD, )
correlationMatrixReduced

array([[ 1.        ,  0.04514676, -0.12184271, ..., -0.03207761,
        -0.04330619, -0.03391959],
       [ 0.04514676,  1.        ,  0.11128586, ...,  0.05446879,
        -0.01017322,  0.05816478],
       [-0.12184271,  0.11128586,  1.        , ...,  0.13475348,
        -0.07433494, -0.07981804],
       ...,
       [-0.03207761,  0.05446879,  0.13475348, ...,  1.        ,
        -0.0038355 , -0.03708557],
       [-0.04330619, -0.01017322, -0.07433494, ..., -0.0038355 ,
         1.        ,  0.95449016],
       [-0.03391959,  0.05816478, -0.07981804, ..., -0.03708557,
         0.95449016,  1.        ]])

In [249]:
correlationRecommendReduced = correlationMatrixReduced[idSubway]
print(correlationRecommendReduced.shape)
print(correlationRecommendReduced)

(129,)
[-0.03602434 -0.23481722 -0.10040932  0.01793079 -0.05869319 -0.01018214
 -0.02981529 -0.00979703 -0.07780665 -0.01960738 -0.02048897  0.73833296
 -0.00183436 -0.05513242 -0.02048907 -0.03312135 -0.04385997 -0.00448809
 -0.00452969 -0.06944352 -0.10483174  0.04644896  0.02665231  0.03692843
 -0.02541816 -0.06192549 -0.02121939 -0.02528444 -0.00479267  0.01102943
 -0.04914154  0.01261178  0.07348972 -0.05434129  0.03448823 -0.0720798
 -0.03822253  0.74822676  0.32265848 -0.02557008 -0.03018736  0.5619614
  0.03121097 -0.03346253  0.00386958  0.58552241  0.58271994 -0.05898611
 -0.0410869  -0.02237872 -0.03217557  0.08096765 -0.05669525 -0.02426094
  0.03279983  0.20456721 -0.00716231 -0.0548328   0.84158877 -0.10212661
  0.84490718 -0.0338506  -0.01383015 -0.05556778  0.52099215 -0.02211963
 -0.00997623 -0.0440739  -0.03686922 -0.03302898 -0.12771961  0.01462413
 -0.0244195  -0.01858985  0.04182561 -0.05927814 -0.07453949 -0.02407751
  0.03937527 -0.01467342 -0.04752109 -0.034294

In [250]:
# Subway
lowLimit = 0.55
upLimit = 0.99

selected_names = names[(correlationRecommendReduced > lowLimit) & (correlationRecommendReduced < upLimit)]
sorted_data = sorted(zip(selected_names, correlationRecommendReduced[(correlationRecommendReduced > lowLimit) & (correlationRecommendReduced < upLimit)]), key=lambda x: x[1], reverse=True)

max_name_length = max(len(name) for name, _ in sorted_data)

for name, correlation in sorted_data:
    print(f"Nombre: {name:{max_name_length}} Correlacion: {correlation:.2f}")

Nombre: Rincon del Bife                                      Correlacion: 0.92
Nombre: Restaurant and Bar and Clothesline Carlos N Charlies Correlacion: 0.84
Nombre: Restaurant Teely                                     Correlacion: 0.84
Nombre: Vips                                                 Correlacion: 0.80
Nombre: Log Yin                                              Correlacion: 0.75
Nombre: Chilis Cuernavaca                                    Correlacion: 0.74
Nombre: Sanborns Casa Piedra                                 Correlacion: 0.70
Nombre: Mikasa                                               Correlacion: 0.59
Nombre: Paniroles                                            Correlacion: 0.58
Nombre: Mariscos Tia Licha                                   Correlacion: 0.56


#**Ejercicio-7.**

Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluación dada al servicio (service_rating). Incluye tus conclusiones.

In [251]:
realtionMatrix = pd.merge(userRestaurantIds, restaurantSpecs[['name','placeID']], on='placeID')   # dataFrame

realtionMatrix.head()

,userID,placeID,rating,food_rating,service_rating,name
0,U1077,135085,2,2,2,Tortas Locas Hipocampo
1,U1108,135085,1,2,1,Tortas Locas Hipocampo
2,U1081,135085,1,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,2,Tortas Locas Hipocampo
4,U1134,135085,2,1,2,Tortas Locas Hipocampo


In [252]:
utilityMatrix = realtionMatrix.pivot_table(values='service_rating', index='userID', columns='name', fill_value=0)
utilityMatrix.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Revisamos relacion de datos nulos con la totalidad de datos

In [253]:
print('Total de elementos de la matriz de utilidad: %d' % (utilityMatrix.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(utilityMatrix)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(utilityMatrix) / utilityMatrix.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 846
Porcentaje de elementos diferentes de cero: 4.8%


In [254]:
SVD = TruncatedSVD(n_components=129, random_state=1)
SVD.fit(X)
completeSVD = SVD.fit_transform(X)
print('Cantidad de valores singulares obtenidos: ',len(SVD.singular_values_))

Cantidad de valores singulares obtenidos:  129


In [255]:
completCorrelationMatrix = np.corrcoef(completeSVD, )
completCorrelationMatrix.shape

(129, 129)

In [256]:
subway = "Subway"

names = utilityMatrix.columns
namesList = list(names)
idSubway = namesList.index(subway)

idSubway

88

In [257]:
correlationRecomendations = completCorrelationMatrix[idSubway]
print(correlationRecomendations.shape)
print(correlationRecomendations)

(129,)
[-7.69028369e-04 -7.87822168e-05 -5.27330975e-03  8.58511349e-04
 -3.03891148e-03  3.97495832e-05 -1.54820208e-03 -5.29347145e-03
  7.27520383e-04 -1.55098452e-03 -1.84483094e-03  3.01090491e-01
 -3.06938616e-05 -3.66639157e-03 -2.28024815e-03 -1.70971180e-03
  2.66961340e-03  4.58467365e-04 -2.11449533e-03 -6.20445134e-03
  3.34268563e-03  1.16598607e-01  1.87080102e-03  2.53682556e-03
 -5.48801141e-03 -2.91288266e-03  2.02825342e-03 -1.36607265e-03
 -2.01695899e-03  1.12058244e-01 -2.36235727e-03  6.52159239e-04
  3.53495770e-03 -3.62793330e-03  1.69065407e-03 -4.51837401e-03
 -2.09301342e-03  2.34634298e-01 -3.80413494e-03 -5.11845710e-04
 -2.27148704e-03  3.88178002e-01 -2.07427108e-03 -2.29428835e-03
  1.32565455e-03  3.01002479e-01  1.71433518e-01 -3.29715354e-03
 -4.74480596e-03 -8.02527884e-04 -2.65071452e-03  1.12018889e-01
 -3.91080724e-03 -2.06369812e-03  1.16008629e-01  2.27855804e-01
  4.28342495e-04 -8.69739761e-04  6.09520875e-01 -6.43364064e-03
  4.25467169e-01 -

In [258]:
# Ponemos un limite superior de 0.99 para que no correlacione tambien el propio
# Subway
lowLimit = 0.23
upLimit = 0.99

selectedNames = names[(correlationRecomendations > lowLimit) & (correlationRecomendations < upLimit)]
sortedData = sorted(zip(selectedNames, correlationRecomendations[(correlationRecomendations > lowLimit) & (correlationRecomendations < upLimit)]), key=lambda x: x[1], reverse=True)

maxNameLength = max(len(name) for name, _ in sortedData)

for name, correlation in sortedData:
    print(f"Nombre: {name:{maxNameLength}} Correlacion: {correlation:.2f}")

Nombre: Restaurant Teely                                     Correlacion: 0.61
Nombre: Vips                                                 Correlacion: 0.51
Nombre: Rincon del Bife                                      Correlacion: 0.43
Nombre: Restaurant and Bar and Clothesline Carlos N Charlies Correlacion: 0.43
Nombre: Sanborns Casa Piedra                                 Correlacion: 0.42
Nombre: Mariscos Tia Licha                                   Correlacion: 0.39
Nombre: Chilis Cuernavaca                                    Correlacion: 0.30
Nombre: Mikasa                                               Correlacion: 0.30
Nombre: Restaurant los Pinos                                 Correlacion: 0.29
Nombre: Log Yin                                              Correlacion: 0.23


In [259]:
num_sv = 30   # En particular los 30 primeros vectores y valores singulares de mayor magnitud tienen esta cantidad de información.

print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Cantidad de información simplificada con los primeros 30 vectores singulares:
47.0%


In [260]:
#Definimos una matriz con una cantidad de 10 vectores solamente
SVD = TruncatedSVD(n_components=num_sv, random_state=1)
truncatedSVD = SVD.fit_transform(X)
truncatedSVD.shape

(129, 30)

In [261]:
correlationMatrixReduced = np.corrcoef(truncatedSVD, )
correlationMatrixReduced

array([[ 1.        ,  0.04514676, -0.12184271, ..., -0.03207761,
        -0.04330619, -0.03391959],
       [ 0.04514676,  1.        ,  0.11128586, ...,  0.05446879,
        -0.01017322,  0.05816478],
       [-0.12184271,  0.11128586,  1.        , ...,  0.13475348,
        -0.07433494, -0.07981804],
       ...,
       [-0.03207761,  0.05446879,  0.13475348, ...,  1.        ,
        -0.0038355 , -0.03708557],
       [-0.04330619, -0.01017322, -0.07433494, ..., -0.0038355 ,
         1.        ,  0.95449016],
       [-0.03391959,  0.05816478, -0.07981804, ..., -0.03708557,
         0.95449016,  1.        ]])

In [262]:
correlationRecommendReduced = correlationMatrixReduced[idSubway]
print(correlationRecommendReduced.shape)
print(correlationRecommendReduced)

(129,)
[-0.03602434 -0.23481722 -0.10040932  0.01793079 -0.05869319 -0.01018214
 -0.02981529 -0.00979703 -0.07780665 -0.01960738 -0.02048897  0.73833296
 -0.00183436 -0.05513242 -0.02048907 -0.03312135 -0.04385997 -0.00448809
 -0.00452969 -0.06944352 -0.10483174  0.04644896  0.02665231  0.03692843
 -0.02541816 -0.06192549 -0.02121939 -0.02528444 -0.00479267  0.01102943
 -0.04914154  0.01261178  0.07348972 -0.05434129  0.03448823 -0.0720798
 -0.03822253  0.74822676  0.32265848 -0.02557008 -0.03018736  0.5619614
  0.03121097 -0.03346253  0.00386958  0.58552241  0.58271994 -0.05898611
 -0.0410869  -0.02237872 -0.03217557  0.08096765 -0.05669525 -0.02426094
  0.03279983  0.20456721 -0.00716231 -0.0548328   0.84158877 -0.10212661
  0.84490718 -0.0338506  -0.01383015 -0.05556778  0.52099215 -0.02211963
 -0.00997623 -0.0440739  -0.03686922 -0.03302898 -0.12771961  0.01462413
 -0.0244195  -0.01858985  0.04182561 -0.05927814 -0.07453949 -0.02407751
  0.03937527 -0.01467342 -0.04752109 -0.034294

In [263]:
# Subway
lowLimit = 0.55
upLimit = 0.99

selected_names = names[(correlationRecommendReduced > lowLimit) & (correlationRecommendReduced < upLimit)]
sorted_data = sorted(zip(selected_names, correlationRecommendReduced[(correlationRecommendReduced > lowLimit) & (correlationRecommendReduced < upLimit)]), key=lambda x: x[1], reverse=True)

max_name_length = max(len(name) for name, _ in sorted_data)

for name, correlation in sorted_data:
    print(f"Nombre: {name:{max_name_length}} Correlacion: {correlation:.2f}")

Nombre: Rincon del Bife                                      Correlacion: 0.92
Nombre: Restaurant and Bar and Clothesline Carlos N Charlies Correlacion: 0.84
Nombre: Restaurant Teely                                     Correlacion: 0.84
Nombre: Vips                                                 Correlacion: 0.80
Nombre: Log Yin                                              Correlacion: 0.75
Nombre: Chilis Cuernavaca                                    Correlacion: 0.74
Nombre: Sanborns Casa Piedra                                 Correlacion: 0.70
Nombre: Mikasa                                               Correlacion: 0.59
Nombre: Paniroles                                            Correlacion: 0.58
Nombre: Mariscos Tia Licha                                   Correlacion: 0.56


#**Ejercicio-8.**

In [264]:
None

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**